In [1]:
import re
import math
import fitz
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.matcher import Matcher
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
import es_core_news_sm
from nltk.tokenize import sent_tokenize
import os
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import es_core_news_md
import itertools
from nltk.stem import SnowballStemmer
import textacy
import regex
import unidecode
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
import re
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)

re_c = re.compile(r'\w+')
wordvectors_file_vec ='/home/erwin/Genoma/cv-parser/parser/embeddings/fasttext-sbwc.3.6.e20.vec'
nlp = es_core_news_md.load()
cantidad = 500000

#model = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [8]:
#file= '/home/erwin/Genoma/cv-parser/parser/resumes_pdf/prueba_genoma_experiencia.pdf'
file ='/home/erwin/Genoma/cv-parser/parser/resumes_pdf/1574824594718-Ing._Comercial_Paula_Fetis_CV.pdf'
#file ='/home/erwin/Genoma/cv-parser/resumes/1580248761709-CV_Sergio_Soto_Valdes.pdf'
#file = '/home/erwin/Genoma/cv-parser/parser/resumes_pdf/1582718980347-CV_-_Mario_Espinoza1.pdf'




In [9]:
def extract_text(path):
    '''
    Input: ruta hacia los archivos
    Salida: Texto plano como string
    Primero se extrae el texto usando fitz (PyMUPDF)
    Luego, debido a que la salida no es perfecta, se limpia
    eliminando todos los simbolos innecesarios.
    Además para efectos de deteccion de nombres, 
    palabras completamente en mayusculas se capitalizan
    sólo al principio. PAULA ---> Paula.
    Esto hace más robusto la detección de entidades.
    '''
    with fitz.open(path) as doc:
        text = ""
        for page in doc: # text contiene todo el texto extraido desde el PDF
            text += page.getText()
        
        text_2 = '' #texto sin mayusculas y saltos innecesarias 
        for line in text.splitlines():
            if not line.strip(): #si la linea esta vacia, saltar
                continue
            line_2=''
            for word in line.split():
                if word.isupper(): # Si la palabra esta completamente en mayuscula
                    line_2 += word.capitalize()+' '

                else:
                    line_2 += word+ ' '
            text_2 += " ".join(line_2.split()) +'\n'
        
        simbolos = ' -,\n./@' #Simbolos que se permiten, sirven para correo, links, etc.
        text_clean = ' '
        for char in text_2:
            if (char.isalnum())| (char in simbolos): #Si el char es alphanumerico o es un simbolo permitido
                text_clean += char
       
        text = text_clean
    return text

text = extract_text(file)
print(text)

 Paula Alejandra Fetis Carrasco
Rut 18.793.163-0
Fecha de nacimiento 05/09/1994
Dirección Warren Smith 70
Ciudad, País Santiago, Chile
Teléfono 56 9 98165994
e-mail p.fetis@gmail.com
Resumen Del Perfil Profesional
Ingeniero Comercial de la Universidad del Desarrollo, con buen manejo de inglés e interés
en área de innovación, finanzas y marketing. Me destaco por mi alta orientación al trabajo
en equipo, liderazgo, obtención de resultados y motivación.
Experiencia Laboral
Mayo 2019 a
Agosto 2019
Rovin Santiago, Chile
Área administrativa / Apoyo back office
 Analizar comportamiento de gastos mensuales
 Control de stock e instalaciones mensuales
Julio 2018 a Abril
2019
Banco Itaú Corpbanca Santiago, Chile
Área comercial
 Parte del programa de jóvenes profesionales, el cual consta de
entrenamiento para área como ejecutiva de cuentas de la
banca preferencial.
 Asesoría y evaluación de distintos productos y servicios
bancarios de clientes.
 Manejo de cartera con aproximadamente 290 clientes.


In [10]:
nlp = es_core_news_sm.load()
nlp_t = nlp(text)

noun_chunks = list(nlp_t.noun_chunks)
print(noun_chunks[1])

País Santiago


In [11]:
# Se cargan los nombres de las secciones, se clasifican en 
# Perfil, Experiencia, Formación académica y skills

seccion_csv = '/home/erwin/Genoma/cv-parser/parser/CSVs/seccionesCV.csv'
secciones = pd.read_csv(seccion_csv, header = 0)
secciones.columns = secciones.loc[0] 
#secciones.columns


secciones_dict = {
    'extras' : [str(x.lower()) for x in secciones.Perfil.values if str(x)!= 'nan'],
    'experiencia' : [str(x.lower()) for x in secciones['Experiencia '].values if str(x)!= 'nan'],
    'educación' : [str(x.lower()) for x in secciones['Formación Académica'].values if str(x)!= 'nan'],
    'skills' : [str(x.lower()) for x in secciones['skills'].values if str(x)!= 'nan']                   
        
}

In [12]:
# switch for debug
flag_print = False

# switch to clear existing data
flag_clear = True

#threshold value for determining section
threshold = 0.45

similar_to = secciones_dict


list_of_sections = similar_to.keys()

# Usando secciones_dict que tiene las secciones a buscar y palabras que describen esas secciones
# se llevan aquellas palabras a su lema
for section in list_of_sections:
    new_list = []
    
    for word in similar_to[section]:
        docx = nlp(word)
        new_list.append(docx[0].lemma_)
    
        
    similar_to[section] = list(set(new_list)) # se retorna lista de elementos unicos
#pp.pprint(similar_to)


# function to remove unnecessary symbols and stopwords 
def modify(word):
    try:
        symbols = '''~'`!@#$%^&*)(_+-=}{][|\:;",./<>?'''
        mod_word = ''
        
        for char in word:
            if (char not in symbols):
                mod_word += char.lower()

        docx = nlp(mod_word)

        if (len(mod_word) == 0 or docx[0].is_stop):
            return None
        else:
            return docx[0].lemma_
    except:
        return None # to handle the odd case of characters like 'x02', etc.
    

    

def is_empty(line):
    '''
    Retorna un booleano correspondiendo a 
    si una linea esta vacia en términos de letras-números
    '''
    for c in line:
        if (c.isalpha()):
            return False
    return True
      

In [13]:
# Se crea un diccionario vacio para rellenarlo
secciones_data = {
    'extras' : '',
    'experiencia' : '',
    'educación' : '',
    'skills':''
                    
        
}


# Se carga un archivo .txt, que contiene el CV que venia del PDF

#file = "resumes_text_output/1580573732905-DiegoAlbertoGutierrezBejarano"
#file = 'resumes_text_output/prueba_genoma_experiencia'
file = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_text/1581878742461-Curriculum_Joselyn_Mari_2020_1'
cv_txt = open(file, "r")
previous_section  = 'extras'



for line in cv_txt:
    # si la linea esta vacia, entonces saltar
    if (len(line.strip()) == 0 or is_empty(line)):
        continue

    # procesar la siguiente linea
    list_of_words_in_line = re_c.findall(line)
    list_of_imp_words_in_line  = []
    
    # recorrer todas las palabras de linea actual
    for i in range(len(list_of_words_in_line)):
        modified_word = modify(list_of_words_in_line[i])

        if (modified_word): 
            list_of_imp_words_in_line.append(modified_word)

    curr_line = ' '.join(list_of_imp_words_in_line)
    doc = nlp(curr_line)
    #print(doc)
    section_value = {}

    # initializing section values to zero
    for section in list_of_sections:
        section_value[section] = 0.0
    section_value[None] = 0.0

    # updating section values    
    for token in doc:
        for section in list_of_sections:
            for word in similar_to[section]:
                #word_token = doc.vocab[word]
                try:
                    section_value[section] = max(section_value[section], float(model.similarity(token.text, word)))
                except: 
                    pass # si es que token.text no esta en el vocabulario
                
    # ver la siguiente sección de acuerdo al umbral establecido
    most_likely_section = None
    for section in list_of_sections:
        if (section_value[most_likely_section] < section_value[section] and section_value[section] > threshold):
            most_likely_section = section

    # updating the section
    if (previous_section != most_likely_section and most_likely_section is not None):
        previous_section = most_likely_section


    # writing data to the pandas series
    try:
        docx = nlp(line)
    except:
        continue  # si que hay simbolos raros
    mod_line = ''
    for token in docx:
        if (not token.is_stop):
            mod_line += token.lemma_ + ' '

    secciones_data[previous_section] += mod_line


cv_txt.close()
    


FileNotFoundError: [Errno 2] No such file or directory: '/home/erwin/Genoma/cv-parser/parser/Outputs/output_text/1581878742461-Curriculum_Joselyn_Mari_2020_1'

In [ ]:
name = file.replace("resumes_text_output/", '').replace('.txt', '')

with open('output_seccionizado/'+name+'.json', 'w',encoding='utf-8') as json_file:
    json.dump(secciones_data, json_file,ensure_ascii=False)
